In [1]:
import os
import time
import datetime
import urllib2
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs

In [2]:
def injury_url_2_df(link):
    test = urllib2.urlopen(link).read()
    final = bs(test, "lxml")

    holder = list()
    for i in final.find_all("table", class_="datatable center"):
        holder.append(i.text.split("\n"))

    columns = holder[0][:7]
    data = holder[0][9:]

    i=0
    new_list=[]
    while i<len(data):
        new_list.append(data[i:i+7])
        i+=7

    df = pd.DataFrame(new_list)[[0,1,2,3,4]]
    df.columns = ["date", "team", 'acquired', 'relinquished', 'notes']
    return df

In [4]:
mth = datetime.datetime.today().month
mth = 12
mth

12

In [5]:
link = "http://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2017-{}-01&EndDate=&ILChkBx=yes&InjuriesChkBx=yes&PersonalChkBx=yes&DisciplinaryChkBx=yes&Submit=Search&start=".format(mth)

In [6]:
# base = pd.DataFrame()
# Shifting to have a base, and scraping just that month's data
base = pd.read_csv("injury_list_2017.csv")
try:
    for i in range(0, 1000, 25):
        print("On results " + str(i))
        base = base.append(injury_url_2_df(link + str(i)))
        time.sleep(1)
except:
    print("Hit end of page! We are done!")

On results 0
On results 25
On results 50
On results 75
On results 100
On results 125
On results 150
On results 175
On results 200
On results 225
On results 250
On results 275
On results 300
On results 325
On results 350
On results 375
On results 400
On results 425
On results 450
On results 475
On results 500
On results 525
On results 550
On results 575
On results 600
On results 625
On results 650
Hit end of page! We are done!


In [7]:
base = base.drop_duplicates()
base.shape

/Users/cliff/anaconda/lib/python2.7/site-packages/pandas/core/algorithms.py:560: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  labels = table.get_labels(values, uniques, 0, na_sentinel, check_nulls)


(1905, 5)

In [8]:
base.to_csv("injury_list_2017.csv", index=False, encoding='utf-8')

### Start from afresh 

In [8]:
base = pd.read_csv("injury_list_2017.csv")
base = base.reset_index(drop=True)
base['acquired'] = [i[5:] for i in base['acquired']]
base['relinquished'] = [i[5:] for i in base['relinquished']]
base['date'] = pd.to_datetime(base['date'], format="%Y-%m-%d")

#### Complicated part is linking injury to recover
- **Injury & recovery may not be 1-1.**
- **Date matching is difficult as they have to be chained together based on recency.**

In [9]:
injury = base[base['relinquished']!=""][["date", 'team', 'relinquished', "notes"]]
injury.rename(columns={'relinquished':'player'}, inplace=True)
injury['status'] = "injuried"

recovered = base[base['acquired']!=""][["date", 'team', 'acquired', "notes"]]
recovered.rename(columns={'acquired':'player'}, inplace=True)
recovered['status'] = "recovered"

### Creating operational injury list
- **Existing workflow is convolated because I want to create a list that I can do backfilling with.**
- **I will sort them by player names & dates. Then, I take the latest status of the player by their order.** 
- **When I wanna do backfilling, I will chop off the date **

In [10]:
c_injury_list = injury.append(recovered).sort_values(["player", "date"])
c_injury_list = c_injury_list[~c_injury_list.notes.str.contains("fined")]

In [11]:
testing_date = (datetime.datetime.today() - datetime.timedelta(days=2))
date = str(testing_date.date())
date

'2017-12-18'

In [12]:
c_injury_list = c_injury_list[c_injury_list['date'] <= date]
c_injury_list = c_injury_list.groupby(['player']).last().reset_index()

# Only keep those that are still injuried. If their last status is recovered, I can remove them!
final = c_injury_list[c_injury_list.status=='injuried']

### Final operational injury list 

In [13]:
final.shape

(144, 5)

In [14]:
final.tail()

,player,date,team,notes,status
326,Wade Baldwin IV,2017-10-20,Blazers,placed on IL with surgery on right thumb,injuried
327,Wayne Selden Jr.,2017-11-18,Grizzlies,placed on IL with sore right quadriceps,injuried
332,Yakuba Ouattara / Billy Ouattara,2017-12-09,Nets,sore left leg (DTD),injuried
334,Zach LaVine,2017-10-19,Bulls,placed on IL recovering from torn ACL in left...,injuried
336,Zaur Pachulia / Zaza Pachulia,2017-12-11,Warriors,placed on IL with sore left shoulder,injuried


In [15]:
final.date.max()

Timestamp('2017-12-16 00:00:00')

### Part 2 -> Player's team list
- **I can just take main_player files from the latest game!**

In [16]:
import glob
glob.glob("*.csv")

['2017_2018_nba_schedule.csv',
 'city_ll_v1.csv',
 'dropbox.csv',
 'game_date_2017.csv',
 'injury_list_2017.csv',
 'main_players_2017.csv',
 'more_home_away_2017.csv',
 'more_team_stats_2017.csv',
 'more_wins_losses_2017.csv',
 'player_season_statistics.csv',
 'referees_2017.csv',
 'start_bench_2017.csv',
 'team_2017.csv']

In [151]:
players = pd.read_csv('main_players_2017.csv')
games = pd.read_csv('game_date_2017.csv')

### Games to predict 

In [202]:
test_date = (testing_date).strftime("%A, %B %-d")
test_date

'Monday, December 4'

In [214]:
games_to_predict = pd.read_csv("2017_2018_nba_schedule.csv")
games = games_to_predict[games_to_predict.date==test_date]
games["away"] = [i.split(" ")[-1] for i in games['away']]
games["home"] = [i.split(" ")[-1] for i in games['home']]
games['away'].replace("Blazers", "Trail Blazers", inplace=True)
games['home'].replace("Blazers", "Trail Blazers", inplace=True)
teams_to_play = games['away'].tolist() + games['home'].tolist()

/Users/cliff/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/cliff/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/cliff/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:3924: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats

In [221]:
teams = pd.read_csv("team_2017.csv")
teams_to_play_ids = teams[teams.TEAM_NAME.isin(teams_to_play)].TEAM_ID.unique()

#### Logic --> Team's latest set of players 

In [239]:
players_to_play = players[players.GAME_ID.isin(players[players.TEAM_ID.isin(teams_to_play_ids)].groupby(["TEAM_ID"]).last().reset_index().GAME_ID.tolist())]
players_to_play.shape

(322, 28)

In [240]:
players_to_play.head(2)

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
8602,21700339,1610612753,ORL,Orlando,203613,Jonathon Simmons,F,NaN,34:17,7.0,...,2.0,4.0,6.0,1.0,1.0,0.0,4.0,4.0,15.0,-9.0
8603,21700339,1610612753,ORL,Orlando,203932,Aaron Gordon,F,NaN,35:28,4.0,...,0.0,6.0,6.0,3.0,1.0,1.0,2.0,2.0,14.0,-9.0


**I need a way to update players availability thru trade or injury updates!**

### Players stats to match 

In [149]:
player_season_stats = pd.read_csv("player_season_statistics.csv")
pstats16 = player_season_stats[player_season_stats.season==2016]
pstats16[pstats16.PLAYER_NAME.isin(test_team_players)].head(2)

,season,PLAYER_ID,PLAYER_NAME,PTS,AST,BLK,REB,STL,games_played,pts_l,...,reb_l,stl_l,type,defenders,facilitator,game_winners,inside_gamers,normal,pure_scorers,useless
10408,2016,201143,Al Horford,14.00000,4.955882,1.264706,6.838235,0.764706,68,0,...,0,0,normal,0,0,0,0,1,0,0
10538,2016,203935,Marcus Smart,10.56962,4.607595,0.430380,3.860759,1.582278,79,0,...,0,1,defenders,1,0,0,0,0,0,0


In [ ]:
tmp = pd.merge(players_to_play, pstats16, on=['PLAYER_ID'], how='left')

In [249]:
tmp[tmp.TEAM_ID==1610612744]

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME_x,START_POSITION,COMMENT,MIN,FGM,...,reb_l,stl_l,type,defenders,facilitator,game_winners,inside_gamers,normal,pure_scorers,useless
172,21700346,1610612744,GSW,Golden State,201142,Kevin Durant,F,NaN,34:36,8.0,...,1.0,0.0,normal,0.0,0.0,0.0,0.0,1.0,0.0,0.0
173,21700346,1610612744,GSW,Golden State,203110,Draymond Green,F,NaN,38:07,6.0,...,1.0,1.0,normal,0.0,0.0,0.0,0.0,1.0,0.0,0.0
174,21700346,1610612744,GSW,Golden State,201580,JaVale McGee,C,NaN,11:06,3.0,...,0.0,0.0,useless,0.0,0.0,0.0,0.0,0.0,0.0,1.0
175,21700346,1610612744,GSW,Golden State,202691,Klay Thompson,G,NaN,37:01,8.0,...,0.0,0.0,pure_scorers,0.0,0.0,0.0,0.0,0.0,1.0,0.0
176,21700346,1610612744,GSW,Golden State,201939,Stephen Curry,G,NaN,34:27,9.0,...,0.0,1.0,normal,0.0,0.0,0.0,0.0,1.0,0.0,0.0
177,21700346,1610612744,GSW,Golden State,1628395,Jordan Bell,NaN,NaN,12:25,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,21700346,1610612744,GSW,Golden State,2738,Andre Iguodala,NaN,NaN,26:04,2.0,...,0.0,0.0,useless,0.0,0.0,0.0,0.0,0.0,0.0,1.0
179,21700346,1610612744,GSW,Golden State,201956,Omri Casspi,NaN,NaN,11:29,0.0,...,0.0,0.0,useless,0.0,0.0,0.0,0.0,0.0,0.0,1.0
180,21700346,1610612744,GSW,Golden State,1626172,Kevon Looney,NaN,NaN,3:22,0.0,...,0.0,0.0,useless,0.0,0.0,0.0,0.0,0.0,0.0,1.0
181,21700346,1610612744,GSW,Golden State,1627775,Patrick McCaw,NaN,NaN,4:22,1.0,...,0.0,0.0,useless,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [246]:
tmp.groupby(["TEAM_ID", "TEAM_ABBREVIATION"])[
    'defenders', 'facilitator', 'game_winners', 'inside_gamers', 'pure_scorers'].sum().reset_index()

,TEAM_ID,TEAM_ABBREVIATION,defenders,facilitator,game_winners,inside_gamers,pure_scorers
0,1610612737,ATL,1.0,1.0,0.0,0.0,0.0
1,1610612738,BOS,1.0,0.0,1.0,0.0,0.0
2,1610612739,CLE,0.0,0.0,0.0,0.0,0.0
3,1610612740,NOP,1.0,1.0,0.0,0.0,0.0
4,1610612741,CHI,0.0,0.0,0.0,0.0,0.0
5,1610612742,DAL,0.0,1.0,0.0,0.0,1.0
6,1610612743,DEN,1.0,0.0,0.0,1.0,0.0
7,1610612744,GSW,0.0,0.0,0.0,0.0,1.0
8,1610612749,MIL,0.0,0.0,0.0,0.0,0.0
9,1610612750,MIN,0.0,0.0,0.0,1.0,1.0


## Checkpoint on current workflow
1. **Have a daily script to update injuries** -- Current workflow does entire list, but an incremental one would be better tbh...
2. **Have a list to extract players from previous game, based on existing (pre-defined) date**
3. **Match names with **
4. Potential improvement: **This workflow doesn't include rookies. Might want to have a system that includes crazy rookies, but has to be done on hindsight and requires more hardcore hardcoding**